## Training with frdd-ml-workflow 

Using the TORP dataset, this tutorial notebook using an NSSL github repo for training ML models.

In [ ]:
# Uncomment to clone the repo; though I recommend running this at the terminal window.
#!git clone https://github.com/NOAA-National-Severe-Storms-Laboratory/frdd-ml-workflow

In [ ]:
# The repo has a stable conda/mamba environment 
#!mamba env create -f environment_py30.yml

## Load TORP Dataset 

In [8]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

from torp.io.torp_dataset import TORPDataset, DEFAULT_FEATURES
import matplotlib.pyplot as plt 
import seaborn as sns
from bayeshist import bayesian_histogram, plot_bayesian_histogram
import numpy as np
import pandas as pd

In [15]:
dataset = TORPDataset(dirpath='/work2/mflora/torp_datasets/ML_data',
                      years=[2011, 2013, 2014, 2015, 2016], 
                     )
df = dataset.load_dataframe()
target_variable = 'tornado'

X, y = df[DEFAULT_FEATURES], df[target_variable]

years = df['year']

In [16]:
df

,timestamp,year,month,day,time,unixTime,latitude,longitude,radar,rangeKm,...,Zdr_Gradient_min,Zdr_Gradient_25th_percentile,Zdr_Gradient_median,Zdr_Gradient_75th_percentile,Zdr_MedianFiltered_max,Zdr_MedianFiltered_mean,Zdr_MedianFiltered_min,Zdr_MedianFiltered_25th_percentile,Zdr_MedianFiltered_median,Zdr_MedianFiltered_75th_percentile
0,20110408-2246,2011,4,8,2246,1302302760,36.3284,-97.7910,KVNX,54.853,...,0.000038,0.000460,0.000723,0.001006,6.1875,4.08768,2.0625,3.4375,4.1875,4.6250
1,20110408-2249,2011,4,8,2249,1302302940,36.3693,-97.8419,KVNX,48.569,...,0.000040,0.000908,0.001508,0.002316,7.9375,4.37325,0.0625,2.2500,4.3125,6.1875
2,20110408-2251,2011,4,8,2251,1302303060,36.3284,-97.7910,KVNX,54.853,...,0.000094,0.000968,0.001499,0.002015,7.9375,4.11035,0.5000,3.1875,4.3125,5.0625
3,20110408-2255,2011,4,8,2255,1302303300,36.3284,-97.7910,KVNX,54.853,...,0.000198,0.000740,0.001271,0.001877,7.9375,4.11962,1.2500,3.0000,4.1250,5.0000
4,20110408-2258,2011,4,8,2258,1302303480,36.3991,-97.7809,KVNX,49.025,...,0.000034,0.000747,0.001104,0.001561,5.3125,2.29739,-0.0625,1.2500,2.2500,3.0625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44425,20161229-1445,2016,12,29,1445,1483022700,32.4460,-83.9065,KJGX,57.963,...,0.000018,0.000720,0.001214,0.001827,6.4375,2.15078,-5.7500,1.2500,2.3125,3.4375
44426,20161229-1446,2016,12,29,1446,1483022760,32.3404,-83.7879,KJGX,55.352,...,0.000018,0.000301,0.000563,0.000998,7.9375,1.40331,-1.5000,0.5625,1.1250,1.5000
44427,20161229-1447,2016,12,29,1447,1483022820,32.3404,-83.7879,KJGX,55.352,...,0.000045,0.000238,0.000401,0.000664,6.0000,1.09580,-0.4375,0.6250,1.0625,1.2500
44428,20161229-1449,2016,12,29,1449,1483022940,32.3404,-83.7879,KJGX,55.352,...,0.000026,0.000327,0.000502,0.000746,3.7500,1.53011,-1.2500,1.0625,1.5625,2.0625


## frdd-ml-workflow's TunedEstimator 

This class incorporates ML pipelines, hyperparameter optimization, and calibration all within a cross-validation framework. 



In [27]:
# To import the package, you can either run python setup.py install from within 
# the frdd-ml-workflow or add the package path to your system's paths. We'll opt for 
# later for simplicity

# Note: the name of the package for me is different as it is an earlier version of the code.
import sys, os 
sys.path.append('/home/monte.flora/python_packages/ml_workflow')

from ml_workflow.tuned_estimator import TunedEstimator
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedGroupKFold

In [28]:
def years_to_groups(years, n_splits=5): 
    """Separated different years into a set of groups based on n_splits"""
    df = dates.copy()
    df = df.to_frame()
    
    unique_dates = np.unique(dates.values)
    
    rs = np.random.RandomState(42)
    rs.shuffle(unique_dates)

    df['groups'] = np.zeros(len(dates))
    for i, group in enumerate(np.array_split(unique_dates, n_splits)):
        df.loc[dates.isin(group), 'groups'] = i+1 
        
    groups = df.groups.values
    
    return groups


def get_cv_method(X,y, years):
    """Build a CV approach based 'years' """
    # Determine the example grouping for the cross-validation. 
    groups = dates_to_groups(years, n_splits=5)
    cv = list(StratifiedGroupKFold(n_splits=5).split(X,y,groups)) 
    return cv, groups
    

In [30]:
# Create a hyperparameter grid to search over. In this case, 
# I am searching over hyperparameters from a random forest. 
search_space = {  'n_estimators' : [100,150,300,400,500], 
                'max_depth' : [6,8,10,15,20],
                'max_features' : [5,6,8,10],
                'min_samples_split' : [4,5,8,10,15,20,25,50],
                'min_samples_leaf' : [4,5,8,10,15,20,25,50],
             }

# Initialize the estimator that will be using.
estimator = RandomForestClassifier(n_jobs=100, random_state=30, criterion = 'entropy') 

# 1. Define the components of the pipeline. 
#   These typically include pre-processing procedures 
#   such as imputation, resampling, scaling, etc.

pipeline_kwargs = {
    # Imputation : How do handle missing values? 
    # -> 'simple' : replace missing with a default value
    'imputer' : 'simple',
    
    # Whether to apply resampling to the dataset to improve 
    # the base rate for training.
    # 'under' == undersampling the majority class to match the minority class.
    # In this, we set it as None, for no resampling.
    'resample' : None,
    
     # For gradient based methods like neural networks, scaling the inputs is neccesary.
     # Not crucial for a RF, so we set it to None. Otherwise, we can
     # set it as 'standard' for mean-stddev scaling.
    'scaler' : None
} 

# 2. Define the parameters for the hyperparameter optimization (hyperopt)
#    This includes the search space (defined above), 
#    the search method ('optimizer'), number of evaluations (max_evals), 
#    the scorer, n_jobs for parallelization, cross-validation method (cv)
#    and where to store the results (output_fname)

scorer = 'average_precision' # AKA Area under the performance diagram curve (AUPDC)
output_fname = 'hyperopt_results.json'

# Get the CV params 
cv, groups = get_cv_method(X,y,years)

hyperopt_kwargs = {'search_space' : search_space, 
                   
                   # Available options: 'tpe', 'grid_search', 'random_search'
                   # 'tpe' is the fancy Bayesian method. 
                   'optimizer' : 'random_search', 
                   'max_evals' : 5,
                   
                   # For the 'tpe' method, how many iterations must pass
                   # if 1% improvement in the score does not happen. 
                   'patience' : 20, 
                  'scorer' : scorer, 
                  'n_jobs' : 1, 
                  'cv' : cv, 
                  'output_fname' : output_fname 
                      }

# 3. Define the calibration parameters. Such as 
#    the method ('isotonic' or 'platt'), 
#    whether to use an ensemble approach, and the cv method. 
#

calibration_cv_kwargs = {'method' : 'isotonic', 
                                     'ensemble' : False, 
                                     'cv' : cv, 
                                     'n_jobs': 1}

model = TunedEstimator(
    estimator,
    pipeline_kwargs,
    hyperopt_kwargs,
    calibration_cv_kwargs,
)

In [31]:
%%time 
model.fit(X, y, groups)

fname = 'random_forest_torp.joblib'

model.save(fname)

/home/monte.flora/python_packages/ml_workflow/ml_workflow/tuned_estimator.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.replace([np.inf, -np.inf], np.nan, inplace=True)
/home/monte.flora/python_packages/ml_workflow/ml_workflow/tuned_estimator.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.replace([np.inf, -np.inf], np.nan, inplace=True)
